## Import libraries

In [1]:
import numpy as np
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Lambda

from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


## Set variables and import data

In [2]:
gen = image.ImageDataGenerator(validation_split=0.2)
batch_size = 64

In [3]:
batches = gen.flow_from_directory('../ISIC/ISIC_2018/data_raw/train',
                                  target_size=(299,299),
                                  class_mode='categorical',
                                  shuffle=True,
                                  batch_size=batch_size,
                                  subset='training')

Found 6654 images belonging to 7 classes.


In [4]:
val_batches = gen.flow_from_directory('../ISIC/ISIC_2018/data_raw/train',
                                  target_size=(299,299),
                                  class_mode='categorical',
                                  shuffle=True,
                                  batch_size=batch_size,
                                  subset='validation')

Found 1660 images belonging to 7 classes.


## Import, configure, and compile model

In [5]:
inception = InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=(299,299,3), pooling=None)

x = inception.layers[-2].output
output_layer = Dense(7, activation='softmax', name='predictions')(x)

inception2 = Model(inputs=inception.input, outputs=output_layer)

inception2.compile(optimizer=Adam(lr=0.00003),
                loss='categorical_crossentropy', metrics=['accuracy'])

## Train model

In [6]:
inception2.fit_generator(batches,
                   steps_per_epoch = batches.samples // batch_size,
                   validation_data = val_batches, 
                   validation_steps = val_batches.samples // batch_size,
                   epochs = 1)

Epoch 1/1
103/103 [==============================] - 79s 763ms/step - loss: 1.1129 - acc: 0.6203 - val_loss: 0.7119 - val_acc: 0.7428
